# Example to Read / Write to Microsoft Hive with Spark

Documentation: https://docs.microsoft.com/en-us/sql/connect/spark/connector?view=sql-server-ver15

An Improvement on the base JDBC connector.


In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
# SQLSERVRER CONFIGURATION
hive_server = "jdbc:hive2://hive-server:10000"


In [3]:
###       .config("spark.jars.packages","com.microsoft.azure:spark-mssql-connector_2.12_3.0:1.0.0-alpha,com.microsoft.sqlserver:mssql-jdbc:9.4.0.jre11")\

# Spark init
spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .enableHiveSupport()\
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

21/11/17 00:20:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/17 00:20:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/17 00:20:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/11/17 00:20:34 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
21/11/17 00:20:34 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
21/11/17 00:20:34 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
21/11/17 00:20:34 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
21/11/17 00:20:34 WARN Utils: Service 'SparkUI' could

In [6]:
spark.sql("show tables;")

DataFrame[database: string, tableName: string, isTemporary: boolean]

In [7]:
# read local data
df = spark.read.option("multiline","true").json("/home/jovyan/datasets/json-samples/stocks.json")
df.toPandas()

,price,symbol
0,126.82,AAPL
1,3098.12,AMZN
2,251.11,FB
3,1725.05,GOOG
4,128.39,IBM
5,212.55,MSFT
6,78.00,NET
7,497.00,NFLX
8,823.80,TSLA
9,45.11,TWTR


In [8]:
# Write create the table on write:
print(hive_server)
df.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", hive_server) \
    .option("dbtable", "default.stocks") \
    .save()

jdbc:hive2://hive-server:10000


Py4JJavaError: An error occurred while calling o63.save.
: org.apache.hive.service.cli.HiveSQLException: Error while compiling statement: FAILED: ParseException line 1:29 cannot recognize input near '"price"' 'DOUBLE' 'PRECISION' in column name or primary key or foreign key
	at org.apache.hive.jdbc.Utils.verifySuccess(Utils.java:267)
	at org.apache.hive.jdbc.Utils.verifySuccessWithInfo(Utils.java:253)
	at org.apache.hive.jdbc.HiveStatement.runAsyncOnServer(HiveStatement.java:313)
	at org.apache.hive.jdbc.HiveStatement.execute(HiveStatement.java:253)
	at org.apache.hive.jdbc.HiveStatement.executeUpdate(HiveStatement.java:490)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.executeStatement(JdbcUtils.scala:993)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.createTable(JdbcUtils.scala:878)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:62)
	at org.apache.spark.sql.execution.datasources.Save

In [19]:
# read back from Mongo
df1 = spark.read.format("mongo") \
    .option("database","example") \
    .option("collection","stocks") \
    .load()
df1.toPandas()

IllegalArgumentException: requirement failed: Missing 'uri' property from options